In [7]:
import xlrd
from pathlib import Path
import pandas as pd
import numpy as np
from numba import decorators
import librosa

import torch
import torchvision
import torchvision.transforms as transforms


In [2]:
# read in the data file
# Give the location of the file 

df = pd.read_excel(r'data.xlsx', sheet_name='reduced totals')
print(df)

        track id   genre amazement     calmness  power  joyful_activation  \
0              1     NaN         7           30      1                  4   
1              2     NaN         5           35      2                  1   
2              3     NaN         8            3     13                 19   
3              4     NaN         5           32      1                  0   
4              5     NaN         2           14      2                  3   
..           ...     ...       ...          ...    ...                ...   
397          398     NaN         3            3      2                  7   
398          399     NaN         4            8      1                  2   
399          400     NaN         3            1      0                  3   
400  Grand Total     NaN      1119         2561   1531               2145   
401          NaN     NaN       awe  contentment  anger         excitement   

     sadness  
0         15  
1          5  
2          7  
3          6  


In [3]:
audio, sample_rate = librosa.load("SongEmotionDataset/1.mp3", res_type='kaiser_fast')
# [print(x) for x in audio]

#convert audio into 2d array
mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
# mfccsscaled = np.mean(mfccs.T,axis=0)
print(mfccs.shape, audio.shape)
mfccs

/Users/carlyzfeng/launchpad/memories/venv/lib/python3.8/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


(40, 2586) (1323648,)


array([[-4.7718567e+02, -3.5018188e+02, -3.2727213e+02, ...,
        -2.3304880e+02, -1.7282356e+02, -1.7300600e+02],
       [ 5.4867729e+01,  1.2169147e+02,  1.3046588e+02, ...,
         1.5205682e+02,  1.3713069e+02,  1.2564399e+02],
       [ 1.7319386e+01, -6.4998174e+00, -1.1335234e+01, ...,
        -2.4199781e+01, -1.8155302e+01, -1.7164921e+01],
       ...,
       [-2.5546131e+00,  1.4165753e-01,  3.2487426e+00, ...,
        -3.0164802e+00, -3.8806877e+00, -3.8585839e+00],
       [-8.8357747e-01, -2.9985919e+00, -3.2912343e+00, ...,
        -1.2004358e+01, -4.5422392e+00, -3.8063698e+00],
       [-4.1244268e-02, -1.5693712e-01, -3.0035594e+00, ...,
        -9.5676631e-01,  1.4220479e+00, -7.0910168e-01]], dtype=float32)

In [6]:
audio_tensor = torch.tensor(mfccs)
audio_tensor

tensor([[-4.7719e+02, -3.5018e+02, -3.2727e+02,  ..., -2.3305e+02,
         -1.7282e+02, -1.7301e+02],
        [ 5.4868e+01,  1.2169e+02,  1.3047e+02,  ...,  1.5206e+02,
          1.3713e+02,  1.2564e+02],
        [ 1.7319e+01, -6.4998e+00, -1.1335e+01,  ..., -2.4200e+01,
         -1.8155e+01, -1.7165e+01],
        ...,
        [-2.5546e+00,  1.4166e-01,  3.2487e+00,  ..., -3.0165e+00,
         -3.8807e+00, -3.8586e+00],
        [-8.8358e-01, -2.9986e+00, -3.2912e+00,  ..., -1.2004e+01,
         -4.5422e+00, -3.8064e+00],
        [-4.1244e-02, -1.5694e-01, -3.0036e+00,  ..., -9.5677e-01,
          1.4220e+00, -7.0910e-01]])

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv1d(1, 128, 80, 4)
        self.bn1 = nn.BatchNorm1d(128)
        self.pool1 = nn.MaxPool1d(4)
        self.conv2 = nn.Conv1d(128, 128, 3)
        self.bn2 = nn.BatchNorm1d(128)
        self.pool2 = nn.MaxPool1d(4)
        self.conv3 = nn.Conv1d(128, 256, 3)
        self.bn3 = nn.BatchNorm1d(256)
        self.pool3 = nn.MaxPool1d(4)
        self.conv4 = nn.Conv1d(256, 512, 3)
        self.bn4 = nn.BatchNorm1d(512)
        self.pool4 = nn.MaxPool1d(4)
        self.avgPool = nn.AvgPool1d(30) #input should be 512x30 so this outputs a 512x1
        self.fc1 = nn.Linear(512, 5)
        
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(self.bn1(x))
        x = self.pool1(x)
        x = self.conv2(x)
        x = F.relu(self.bn2(x))
        x = self.pool2(x)
        x = self.conv3(x)
        x = F.relu(self.bn3(x))
        x = self.pool3(x)
        x = self.conv4(x)
        x = F.relu(self.bn4(x))
        x = self.pool4(x)
        x = self.avgPool(x)
        x = x.permute(0, 2, 1) #change the 512x1 to 1x512
        x = self.fc1(x)
        return F.log_softmax(x, dim = 2)

model = Net()
model.to(device)
print(model)